For a rich view, please use this link: <a href="https://nbviewer.org/github/0xrsfagundes/Portugal-Stats/blob/master/Most%20Dangerous%20Cities%20Map%20Plot.ipynb">https://nbviewer.org/github/0xrsfagundes/Portugal-Stats/blob/master/Most%20Dangerous%20Cities%20Map%20Plot.ipynb</a>

<h2>Library</h2>

In [1]:
import folium
from folium.plugins import MiniMap
import pandas as pd
import requests
import warnings

warnings.filterwarnings('ignore')

In [2]:
# Bing Maps API key
bing_api_key = 'AqftqirjkuYeZgkG9LEdoTjhJq8d0RuX0oVV-gDt5q2w5hgqtnGcB6BfwJsp29pr'

<h2>Functions</h2>

In [3]:
# Function to customize the marker icon color
def get_marker_color(value):
    if value < 10000:
        return 'green'
    elif value < 15000:
        return 'orange'
    else:
        return 'red'

def create_bing_map_with_cities(city_df):
    # Create a map centered around Portugal
    m = folium.Map(location=[39.3999, -8.2245], zoom_start=6, tiles="cartodb positron")  # Centered on Portugal

    # Add a MiniMap to the main map (optional but useful)
    minimap = MiniMap()
    m.add_child(minimap)

    # Iterate through the DataFrame and add markers for each city
    for index, row in city_df.iterrows():
        # Customize the popup content with city name and value
        popup_html = f"<b>{row['city']}</b><br>Value: {row['value']}"

        # Create a custom icon with the specified marker color
        icon = folium.DivIcon(
            html=f'<div style="color: {get_marker_color(row["value"])};"><i class="fa fa-map-pin"></i></div>',
            icon_size=(30, 30),
        )
        
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=folium.Popup(popup_html, max_width=300),  # Use customized HTML popup
            icon=icon,
        ).add_to(m)

    return m

<h2>Main</h2>

In [4]:
# Load csv file from "Portugal Crimes Report"
df = pd.read_csv('csv/df_cities_crimes.csv')

In [5]:
# Get the Latitude and Longitude using Bing Maps API
# Define the cities to plot
cities = df['city'].unique()

# Create an empty list to store city information dictionaries
city_info_list = []

# Loop through the cities and add a marker for each one
for city in cities:
    # Use the Bing Maps REST Services API to get the latitude and longitude coordinates of the city
    url = 'http://dev.virtualearth.net/REST/v1/Locations'
    params = {'q': city, 'key': bing_api_key}
    response = requests.get(url, params=params).json()

    # Check if the response contains any results
    if 'resourceSets' in response and response['resourceSets']:
        location = response['resourceSets'][0]['resources'][0]['point']['coordinates']

        # Create a dictionary with city information
        city_info = {'city': city, 'latitude': location[0], 'longitude': location[1]}

        # Add the city information dictionary to the list
        city_info_list.append(city_info)

In [6]:
# Convert dictionary into a dataframe
df_map = pd.DataFrame(city_info_list)

df_map.head()

,city,latitude,longitude
0,Oliveira do Bairro,40.515446,-8.494024
1,Sever do Vouga,40.733063,-8.369164
2,Vagos,40.555264,-8.681458
3,Figueira da Foz,40.150784,-8.851213
4,Mealhada,40.378105,-8.451446


In [7]:
# Merge original dataframe with dataframe with coordinates
df_merged = pd.merge(df, df_map, on='city', how='left')

# Create a file with coordinates
df_merged.to_csv('csv/df_cities_crimes_map.csv')

df_merged.head()

,city,crime,value,latitude,longitude
0,Oliveira do Bairro,Roubo por esticão e na via pública,3,40.515446,-8.494024
1,Sever do Vouga,Roubo por esticão e na via pública,4,40.733063,-8.369164
2,Vagos,Roubo por esticão e na via pública,3,40.555264,-8.681458
3,Figueira da Foz,Roubo por esticão e na via pública,19,40.150784,-8.851213
4,Mealhada,Roubo por esticão e na via pública,0,40.378105,-8.451446


In [8]:
# get the 30 most dangerous cities in Portugal
df_most_dangerous_cities = df_merged.groupby(['city','latitude','longitude'])['value'].sum().sort_values(ascending=False).head(30).reset_index()

df_most_dangerous_cities.head()

,city,latitude,longitude,value
0,Algarve,37.014439,-7.935280,68708
1,Lisboa,38.707008,-9.135641,47059
2,Alentejo,38.408916,-7.820642,37583
3,Porto,41.150154,-8.610318,20856
4,Sintra,38.798443,-9.388219,17379


In [9]:
df_most_dangerous_cities['value'].describe()

count       30.000000
mean     14272.133333
std      13700.354397
min       6066.000000
25%       6768.500000
50%       9793.000000
75%      14140.000000
max      68708.000000
Name: value, dtype: float64

In [10]:
create_bing_map_with_cities(df_most_dangerous_cities)